In this Notebook I trained two classifier, one to identify hope and one to identify nostalgia.

# Package Import

In [1]:
# General Packages
import pandas as pd
import numpy as np
import re # for text-cleaning
from google.colab import data_table, files
data_table.enable_dataframe_formatter() # to have tables which enable reading the text in full
import warnings
from joblib import dump, load

# Data Handling
!git config --global credential.helper store # To upload a Model to Huggingface
!sudo apt-get install git-lfs # To upload a Model to Huggingface
#!huggingface-cli login # To upload a Model to Huggingface 

# Machine Learning
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, balanced_accuracy_score
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.naive_bayes import MultinomialNB

# set random seed for reproducibility
SEED_GLOBAL = 1984
np.random.seed(SEED_GLOBAL)

# NLP
import nltk
nltk.download('punkt')
from nltk.tokenize import TreebankWordTokenizer, WhitespaceTokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


# Transformer Packages (Laurer, 2023)
!pip install datasets
!pip install transformers==4.40.0 # in Colab I got an error with the trainer when I did not download the most recent transformer
!pip install accelerate -U
import datasets
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from transformers import pipeline, TrainingArguments, Trainer, logging
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"  # use GPU (cuda) if available, otherwise use CPU



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: write).
Your token has been sa

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 25.0 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.2
    Uninstalling transformers-4.40.2:
      Successfully uninstalled transformers-4.40.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-m

# Data

### Dataset Import
The polnos datasets for Nostalgia expressions are open source. You can download them via the [Harvard Dataverse](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/L198GI). The Polyhope Dataset is permitted to use only or redistribute only for non-commercial or academic-research purposes. It can be downloaded from the [HOPE at IberLEF 2024](https://codalab.lisn.upsaclay.fr/competitions/17714#participate-get_starting_kit) competition.

To make handling easier, I loaded them into my Github and will use them from there.

In [2]:
#https://www.geeksforgeeks.org/how-to-upload-folders-to-google-colab/
!git clone https://github.com/BeJa1996/political_hope_nostalgia/
!unzip political_hope_nostalgia/Training_Datasets.zip

Cloning into 'political_hope_nostalgia'...
remote: Enumerating objects: 32, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 32 (delta 8), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (32/32), 5.83 MiB | 7.80 MiB/s, done.
Resolving deltas: 100% (8/8), done.
Archive:  political_hope_nostalgia/Training_Datasets.zip
   creating: Training_Datasets/
  inflating: Training_Datasets/data_polnos_handcoding.csv  
  inflating: Training_Datasets/Task 2_Test_with_labels_English_PolyHope.csv  
  inflating: Training_Datasets/data_polnos_handcoding_validation.csv  


## Nostalgia Dataset

Müller and Proksch (2023a) created two datasets which I can use to train and validate my classifiers. One is '**data_polnos_handcoding**' and the other is '**data_polnos_handcoding_validation**'. Both come from Müller and Proksch (2023b). In Polnos Handcoding, there are 1200 sentences which where coded by four coders according to whether they contain nostalgia or not. In handcoding validation there are 3515 sentences which were coded as nostalgic by one of their annotation methods and which were manually vaidated by two coders.

Because in the first dataset there are only 219 sentences coded as nostalgic by at least two human corders, I add the sentences from the second dataset, which were also coded by two coders as nostalgic. In this way I also reduce the imbalance of the dataset.

### Dataset Import

In [ ]:
nost_handcoding = pd.read_csv('/content/Training_Datasets/data_polnos_handcoding.csv')
nost_validation = pd.read_csv('/content/Training_Datasets/data_polnos_handcoding_validation.csv')

### Dataset Overview

I want to first inspect the structure and content of the datasets.

#### Handcoding

In [ ]:
nost_handcoding.shape

(1200, 29)

In [ ]:
nost_handcoding.dtypes

doc_id                          object
countryname                     object
party                            int64
manifesto_id                    object
text                            object
cmp_code                        object
nostalgic_at_least_1             int64
nostalgic_at_least_2             int64
nostalgic_at_least_3             int64
nostalgic_at_least_4             int64
translation_at_least_1           int64
translation_at_least_2           int64
translation_at_least_3           int64
translation_at_least_4           int64
nostalgic_coder1                 int64
nostalgic_coder2                 int64
nostalgic_coder3                 int64
nostalgic_coder4                 int64
translation_coder1               int64
translation_coder2               int64
translation_coder3               int64
translation_coder4               int64
translation_agreement_coders     int64
nostalgia_sum                    int64
nostalgia_sum_emb                int64
nostalgia_emb            

Nostalgia_agreement_coders informs about how many coders agreed that a text is be nostalgic. Let us see how the texts differ.

In [ ]:
nost_handcoding.loc[nost_handcoding['nostalgia_agreement_coders'] == 0,
 ['text', 'nostalgia_agreement_coders']].head()

,text,nostalgia_agreement_coders
0,"Economic injustice and cultural, historical an...",0
1,"a new Packaging Act will be adopted, which, am...",0
2,Nationals of countries with which France has n...,0
4,The ULA condemns the complete failure of the g...,0
5,legal certainty - necessary for citizens and i...,0


In [ ]:
nost_handcoding.loc[nost_handcoding['nostalgia_agreement_coders'] == 1,
 ['text', 'nostalgia_agreement_coders']].head()

,text,nostalgia_agreement_coders
8,The NATO returned to the main and most importa...,1
11,Erase from our streets and squares any honorab...,1
29,Which shows the effectiveness of what we do fr...,1
31,Germany is a successful integration of the cou...,1
35,The past four years of work was also made a se...,1


In [ ]:
nost_handcoding.loc[nost_handcoding['nostalgia_agreement_coders'] == 2,
 ['text', 'nostalgia_agreement_coders']].head()

,text,nostalgia_agreement_coders
3,7. The re-emerging LSDP their active involveme...,2
7,Restore bilateral migration strategy cooperati...,2
9,11) Restoring the minimum wage at €8.65 an hour.,2
34,The result has been a mere administrative dece...,2
42,Popular Alliance proposes: - Ending the cyclic...,2


In [ ]:
nost_handcoding.loc[nost_handcoding['nostalgia_agreement_coders'] > 2,
 ['text', 'nostalgia_agreement_coders']].head()

,text,nostalgia_agreement_coders
13,"• Modern, on the personal development focused ...",3
15,We disagree with the globalization that aims t...,4
32,2. Socialist ideas emerged Lithuania Lithuania...,4
41,In the history of civilization at a time when ...,4
86,The course of national history is indispensabl...,4


In [ ]:
print(nost_handcoding.groupby(['nostalgic_at_least_2'])['text'].count())

nostalgic_at_least_2
0    981
1    219
Name: text, dtype: int64


#### Validation

In [ ]:
nost_validation.shape

(3515, 17)

In [ ]:
nost_validation.dtypes

manifesto_id                     object
countryname                      object
text_pre                         object
text                             object
text_post                        object
party                             int64
party_family_recoded             object
cmp_code                         object
nostalgia_sentence_dummy_emb      int64
nostalgia_sentence_bert           int64
nostalgia_sentence_svm            int64
nostalgic_coding_coder1           int64
nostalgic_coding_coder2           int64
nostalgia_coded_both              int64
nostalgia_coded_at_least_one      int64
score_gpt                       float64
justification_gpt                object
dtype: object

In [ ]:
nost_validation.loc[nost_validation['nostalgia_coded_both'] == 1,['text']].head()

,text
7,Here for the first time since independence in ...
14,Strengthen the United Kingdom and protect and ...
16,Danish book rental and ground traffic control ...
23,The culture is not only considered one of the ...
32,Compulsory school age Restore: restoring lasti...


In [ ]:
nost_validation.groupby('nostalgia_coded_both')['nostalgia_coded_both'].count()

nostalgia_coded_both
0    3269
1     246
Name: nostalgia_coded_both, dtype: int64

### Dataset Preparation

I rename columns into label and label test, because that is necessary for the NLI pipeline from Laurer et al., 2023. Then I will concat the two datasets and lowercase the texts.

In [ ]:
nost_handcoding['label'] = np.where(nost_handcoding['nostalgia_agreement_coders'] >= 2, 1,0) # https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/
nost_handcoding['label_text'] = np.where(nost_handcoding['nostalgia_agreement_coders'] >= 2,
                                         'Nostalgia','Not Nostalgia') # https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/
nost_validation['label'] = np.where(nost_validation['nostalgia_coded_both'] == 1, 1,0) # https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/
nost_validation['label_text'] = np.where(nost_validation['nostalgia_coded_both'] == 1,
                                         'Nostalgia','Not Nostalgia') # https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/


In [ ]:
nostalgia = nost_validation.loc[nost_validation['label'] == 1,['text', 'label', 'label_text']]
nostalgia = pd.concat([nostalgia, nost_handcoding[['text', 'label', 'label_text']]])
nostalgia.shape

(1446, 3)

The text does not entail much special characters, so it will probably be enough to lower case it.

In [ ]:
nostalgia['text'] = nostalgia.apply(lambda row: row['text'].lower(),axis=1)

In [ ]:
nostalgia.groupby('label_text')['text'].count()

label_text
Nostalgia        465
Not Nostalgia    981
Name: text, dtype: int64

Train Validation Test Split

In [ ]:
nost_train, nost_test = train_test_split(nostalgia, test_size = .2, stratify = nostalgia['label'] )
nost_test, nost_validation = train_test_split(nost_test, test_size = .5, stratify = nost_test['label'] )

In [ ]:
nost_train.groupby('label_text')['label_text'].count()

label_text
Nostalgia        372
Not Nostalgia    784
Name: label_text, dtype: int64

In [ ]:
nost_test.groupby('label_text')['label_text'].count()

label_text
Nostalgia        46
Not Nostalgia    99
Name: label_text, dtype: int64

In [ ]:
nost_validation.groupby('label_text')['label_text'].count()

label_text
Nostalgia        47
Not Nostalgia    98
Name: label_text, dtype: int64

## Hope
Dataset from *PolyHope: Two-level hope speech detection from tweets*. It can be downloaded through the [HOPE at IberLEF 2024](https://codalab.lisn.upsaclay.fr/competitions/17714#learn_the_details-terms_and_conditions) competition.

**Generalized Hope**
“According to Ezzy (2000), Smith and Sparkes (2005), Particularized hope is similar to the typical definition of hope used in the psychological literature as the expectation and desire for specific events and outcomes (e.g., I hope the surgery will be successful). In contrast, Generalized hope is characterized by openness to events and outcomes (e.g., I hope I will get well).” (Balouchzahi et al., 2023, p. 2)

**Realistic Hope**
“Realistic hope can be described as the hope for a specific outcome, which involves the process of mental imagery and the calculation of the probability of occurrence to prevent the person from losing touch with reality (Webb, 2007)” (Balouchzahi et al., 2023, p. 2)

**Unrealistic Hope**
“In contrast, unrealistic hopes are based on incomplete or incorrect information and hope for something unlikely to happen (Verhaeghe et al., 2007) (e.g., my grades are bad, and everyone says I have failed, but I am waiting for a miracle to happen)” (Balouchzahi et al., 2023, p. 2)

## Dataset import

In [3]:
hope = pd.read_csv('/content/Training_Datasets/Task 2_Test_with_labels_English_PolyHope.csv')

### Dataset Overview

Again. I want to get an overview about the data first.

In [ ]:
hope.shape

(6192, 4)

In [ ]:
hope.dtypes

text          object
binary        object
multiclass    object
id             int64
dtype: object

In [ ]:
hope.groupby('binary')['binary'].count()

binary
Hope        3104
Not Hope    3088
Name: binary, dtype: int64

In [ ]:
hope.groupby('multiclass')['multiclass'].count()

multiclass
Generalized Hope    1726
Not Hope            3088
Realistic Hope       730
Unrealistic Hope     648
Name: multiclass, dtype: int64

In [ ]:
hope[hope['multiclass']== 'Generalized Hope'].head()

,text,binary,multiclass,id
1,#USER# Oh shit really? I would hope they'd she...,Hope,Generalized Hope,4061
2,"#USER# Good morning, Bud! 🥰 Another good decis...",Hope,Generalized Hope,1621
5,#USER# 49ers in the NFL are a private company....,Hope,Generalized Hope,3994
6,$SPY $SPX update:\nLooking excellent. Pretty n...,Hope,Generalized Hope,1780
11,I got baloons😌🌚\n\nA whole year has passed and...,Hope,Generalized Hope,6167


In [ ]:
hope[hope['multiclass']== 'Realistic Hope'].head()

,text,binary,multiclass,id
0,"#USER# #USER# I'm really liking this project, ...",Hope,Realistic Hope,5820
33,Inshallah if I do become a doc and if I leave ...,Hope,Realistic Hope,4720
45,#USER# They're doing it now hoping everyone wi...,Hope,Realistic Hope,58
59,"#USER# #USER# He can pray all he wants, just n...",Hope,Realistic Hope,2599
67,#USER# #USER# hope you guys got d msg from d y...,Hope,Realistic Hope,6513


In [ ]:
hope[hope['multiclass']== 'Unrealistic Hope'].head()

,text,binary,multiclass,id
3,i aspire to have the level of delusion to beli...,Hope,Unrealistic Hope,1754
16,Really wish they ain’t have to cut some of the...,Hope,Unrealistic Hope,6744
17,I wish my bf were less attractive to me like h...,Hope,Unrealistic Hope,2102
28,Wallahi Indian Muslims are the bravest Muslim...,Hope,Unrealistic Hope,3078
30,#USER# The course I wish more people created. ...,Hope,Unrealistic Hope,7739


### Dataset Preparation

In the texts there are many Hashtags and Emojis. I need to remove them, because they will not be there in the youtube content, but they might provide information on which the algorithm learns. I want the algorithm to concentrate only on the features which would be there in the final set as well.

In [4]:
# from https://www.kaggle.com/code/tariqsays/tweets-cleaning-with-python
hope['old_text'] = hope['text']
hope['text'] = hope.apply(lambda row: row['text'].lower(),axis=1)
hope['text'] = hope.apply(lambda row: re.sub("@[A-Za-z0-9_]+","",
                                                     row['text']),axis=1)
hope['text'] = hope.apply(lambda row: re.sub("#[A-Za-z0-9_]+","",
                                                   row['text']),axis=1)
# i need to do it a second time, because anonymization lead to #User# which
# was not removed
hope['text'] = hope.apply(lambda row: re.sub("#","",
                                                   row['text']),axis=1)
hope['text'] = hope.apply(lambda row: re.sub(r"http\S+","",
                                                     row['text']),axis=1)
hope['text'] = hope.apply(lambda row: re.sub(r"www.\S+","",
                                                  row['text']),axis=1)
# in the following I added punctuation, so that this is not removed
hope['text'] = hope.apply(lambda row: re.sub("[^a-z0-9\.,;:]"," ",
                                                     row['text']),axis=1)

hope.head()

,text,binary,multiclass,id,old_text
0,"i m really liking this project, let s work t...",Hope,Realistic Hope,5820,"#USER# #USER# I'm really liking this project, ..."
1,oh shit really i would hope they d shed some...,Hope,Generalized Hope,4061,#USER# Oh shit really? I would hope they'd she...
2,"good morning, bud another good decision fr...",Hope,Generalized Hope,1621,"#USER# Good morning, Bud! 🥰 Another good decis..."
3,i aspire to have the level of delusion to beli...,Hope,Unrealistic Hope,1754,i aspire to have the level of delusion to beli...
4,projects are continuously attacked by hacker...,Not Hope,Not Hope,401,#USER# #USER# Projects are continuously attack...


In [5]:
hope['label'] = np.where(hope['binary'] == 'Hope', 1,0) # https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/
hope['label_text'] = np.where(hope['binary'] == 'Hope', 'Hope','Not Hope') # https://www.dataquest.io/blog/tutorial-add-column-pandas-dataframe-based-on-if-else-condition/
hope_prepared = hope[['text', 'label', 'label_text']]
len(hope_prepared)

6192

In [6]:
hope_prepared = hope_prepared[~hope_prepared.duplicated('text', keep = 'first')]
len(hope_prepared)

6185

Train, Validation, Test Split

In [7]:
hope_train, hope_test_validation = train_test_split(hope_prepared, test_size = .2, stratify = hope_prepared['label'] )
hope_test, hope_validation = train_test_split(hope_test_validation, test_size = .5, stratify = hope_test_validation['label'] )

In [8]:
hope_train.groupby('label_text')['label_text'].count()

label_text
Hope        2480
Not Hope    2468
Name: label_text, dtype: int64

In [9]:
hope_test.groupby('label_text')['label_text'].count()

label_text
Hope        310
Not Hope    308
Name: label_text, dtype: int64

In [10]:
hope_validation.groupby('label_text')['label_text'].count()

label_text
Hope        310
Not Hope    309
Name: label_text, dtype: int64

# Functions


#### nb_training
Grid Search to find the best classifier (from Atteveld et al.,2021).

In [11]:
def nb_training(emotion, train_df):
  pipe = Pipeline( #https://cssbook.net/content/chapter11.html
    steps=[
        ("vectorizer", None),
        ("classifier",  MultinomialNB()),
      ]
  )
  grid = {
      "vectorizer": [CountVectorizer(stop_words='english'),
                    TfidfVectorizer(stop_words='english')],
      "vectorizer__tokenizer": [None,
                              TreebankWordTokenizer().tokenize,
                              WhitespaceTokenizer().tokenize],
      "vectorizer__ngram_range": [(1, 1), (1, 2), (1,3)],
      "vectorizer__max_df": [0.5, 1.0],
      "vectorizer__min_df": [1, 5, 10],
  }

  nb = GridSearchCV(
      estimator=pipe, n_jobs=-1, param_grid=grid, scoring="f1_micro",
      cv=5, refit=True #https://stackoverflow.com/a/71140866
  )
  nb.fit(train_df['text'], train_df['label_text'])

  # Download Model
  # First lines from https://stackoverflow.com/a/71140866
  estimator = nb.best_estimator_
  #dump(estimator, f"{emotion}_nb_model.joblib")
  #files.download(f"{emotion}_nb_model.joblib") # https://stackoverflow.com/a/50192371

  return nb

### clean_memory(
Memory Cleaner (Laurer 2023)

In [12]:
# helper function to clean memory and reduce risk of out-of-memory error
import gc
def clean_memory():
  #del(model)
  if torch.cuda.is_available():
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
  gc.collect()

clean_memory()

### Reformatting Functions
The functions are from [Laurer (2023)]().

In [13]:
## function for reformatting the train set
def format_nli_trainset(df_train=None, hypo_label_dic=None, random_seed=42):
  print(f"Length of df_train before formatting step: {len(df_train)}.")
  length_original_data_train = len(df_train)

  df_train_lst = []
  for label_text, hypothesis in hypo_label_dic.items():
    ## entailment
    df_train_step = df_train[df_train.label_text == label_text].copy(deep=True)
    df_train_step["hypothesis"] = [hypothesis] * len(df_train_step)
    df_train_step["label"] = [0] * len(df_train_step)
    ## not_entailment
    df_train_step_not_entail = df_train[df_train.label_text != label_text].copy(deep=True)
    df_train_step_not_entail = df_train_step_not_entail.sample(n=min(len(df_train_step), len(df_train_step_not_entail)), random_state=random_seed)
    df_train_step_not_entail["hypothesis"] = [hypothesis] * len(df_train_step_not_entail)
    df_train_step_not_entail["label"] = [1] * len(df_train_step_not_entail)
    # append
    df_train_lst.append(pd.concat([df_train_step, df_train_step_not_entail]))
  df_train = pd.concat(df_train_lst)

  # shuffle
  df_train = df_train.sample(frac=1, random_state=random_seed)
  df_train["label"] = df_train.label.apply(int)
  df_train["label_nli_explicit"] = ["True" if label == 0 else "Not-True" for label in df_train["label"]]  # adding this just to simplify readibility

  print(f"After adding not_entailment training examples, the training data was augmented to {len(df_train)} texts.")
  print(f"Max augmentation could be: len(df_train) * 2 = {length_original_data_train*2}. It can also be lower, if there are more entail examples than not-entail for a majority class.")

  return df_train.copy(deep=True)


## function for reformatting the test set
def format_nli_testset(df_test=None, hypo_label_dic=None):
  ## explode test dataset for N hypotheses
  hypothesis_lst = [value for key, value in hypo_label_dic.items()]
  print("Number of hypotheses/classes: ", len(hypothesis_lst))

  # label lists with 0 at alphabetical position of their true hypo, 1 for not-true hypos
  label_text_label_dic_explode = {}
  for key, value in hypo_label_dic.items():
    label_lst = [0 if value == hypo else 1 for hypo in hypothesis_lst]
    label_text_label_dic_explode[key] = label_lst

  df_test["label"] = df_test.label_text.map(label_text_label_dic_explode)
  df_test["hypothesis"] = [hypothesis_lst] * len(df_test)
  print(f"Original test set size: {len(df_test)}")

  # explode dataset to have K-1 additional rows with not_entail label and K-1 other hypotheses
  # ! after exploding, cannot sample anymore, because distorts the order to true label values, which needs to be preserved for evaluation code
  df_test = df_test.explode(["hypothesis", "label"])  # multi-column explode requires pd.__version__ >= '1.3.0'
  print(f"Test set size for NLI classification: {len(df_test)}\n")

  df_test["label_nli_explicit"] = ["True" if label == 0 else "Not-True" for label in df_test["label"]]  # adding this just to simplify readibility

  return df_test.copy(deep=True)


### Metrics
[Laurer 2023]()

In [14]:
# I took the core part out of the functions from Laurer, because I need  it multiple times #bj
def metrics_core (true_labels, predicted_labels):
  precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='macro')  # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
  precision_micro, recall_micro, f1_micro, _ = precision_recall_fscore_support(true_labels, predicted_labels, average='micro')  # https://scikit-learn.org/stable/modules/generated/sklearn.metrics.precision_recall_fscore_support.html
  acc_balanced = balanced_accuracy_score(true_labels, predicted_labels)
  acc_not_balanced = accuracy_score(true_labels, predicted_labels)

  metrics = {
      'accuracy': acc_not_balanced,
      'accuracy_balanced': acc_balanced,
      'f1_macro': f1_macro,
      'f1_micro': f1_micro,
      'precision_macro': precision_macro,
      'recall_macro': recall_macro,
      'precision_micro': precision_micro,
      'recall_micro': recall_micro,
  }
  return metrics

In [15]:
def compute_metrics_standard(eval_pred): # https://github.com/MoritzLaurer/summer-school-transformers-2023/blob/main/3_tune_bert.ipynb
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore")

        labels = eval_pred.label_ids
        pred_logits = eval_pred.predictions
        preds_max = np.argmax(pred_logits, axis=1)  # argmax on each row (axis=1) in the tensor

        # metrics
        metrics = metrics_core(labels, preds_max) # taken into extra function #bj

        print("Detailed metrics: \n",
              classification_report(labels,
                                    preds_max,
                                    digits=2,
                                zero_division='warn'), "\n")

        return metrics


def compute_metrics_nli_binary(eval_pred, label_text_alphabetical=None):
    predictions, labels = eval_pred

    ### reformat model output to enable calculation of standard metrics
    # split in chunks with predictions for each hypothesis for one unique premise
    def chunks(lst, n):  # Yield successive n-sized chunks from lst. https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
        for i in range(0, len(lst), n):
            yield lst[i:i + n]

    # for each chunk/premise, select the most likely hypothesis
    softmax = torch.nn.Softmax(dim=1)
    prediction_chunks_lst = list(chunks(predictions, len(set(label_text_alphabetical)) ))
    hypo_position_highest_prob = []
    for i, chunk in enumerate(prediction_chunks_lst):
        hypo_position_highest_prob.append(np.argmax(np.array(chunk)[:, 0]))  # only accesses the first column of the array, i.e. the entailment/true prediction logit of all hypos and takes the highest one

    label_chunks_lst = list(chunks(labels, len(set(label_text_alphabetical)) ))
    label_position_gold = []
    for chunk in label_chunks_lst:
        label_position_gold.append(np.argmin(chunk))  # argmin to detect the position of the 0 among the 1s

    ### calculate standard metrics
    metrics = metrics_core(label_position_gold, hypo_position_highest_prob) # taken into extra function #bj

    print("Detailed metrics: \n",
          classification_report(label_position_gold,
                                hypo_position_highest_prob,
                                digits=2,
                                zero_division='warn'), "\n")
    return metrics


### model_finetuning
In the following function I consolidate the Fine-Tuning Steps from Laurer (2023) from the NLI and the Non-NLI set. I quoted, If I took code from somewhere else. I mark my comments with (bj)

In [16]:
def model_finetuning(emotion, df_train, df_test, model_name, seed, epochs = 10, nli = False, hypo_label_dict = {} ):
  """
    Trains and Evaluates a Transformer Model from Huggingface

    Returns
    -------
    model
        The path to the optimal model
    results
  """
  clean_memory()
  model_short = model_name.split("/")[1]
  label_text_alphabetical = np.sort(df_train.label_text.unique())

  # Prepare Data (bj)
  print('\nData Preparation and Tokenizer Download\n')
  ## Extra Preparation NLI (bj)
  if nli == True:
      if not hypo_label_dict: # https://stackoverflow.com/a/23177452/23292830
        raise Exception("No Dictionary") #https://www.w3schools.com/python/gloss_python_raise.asp

      df_train = format_nli_trainset(df_train=df_train,
                                           hypo_label_dic=hypo_label_dict,
                                           random_seed=seed)
      df_test = format_nli_testset(df_test=df_test,
                                         hypo_label_dic=hypo_label_dict)
      config = None #bj
  else:
    # i changed the following dictionary creation, because the ids were wrong
    label2id = dict(zip(df_test['label_text'].unique(), df_test['label'].unique().tolist()))
    id2label = dict(zip(df_test['label'].unique().tolist(), df_test['label_text'].unique()))
    config = AutoConfig.from_pretrained(model_name, label2id=label2id, id2label=id2label, num_labels=len(label2id));
    print("\n", label2id, "\n")

  dataset = datasets.DatasetDict({
    "train": datasets.Dataset.from_pandas(df_train),
    "test": datasets.Dataset.from_pandas(df_test)
  })

  # Tokenize (bj)
  tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, model_max_length=512)

  def tokenize(examples):
    if nli == True:
      return tokenizer(examples["text"], examples["hypothesis"], truncation=True, max_length=512)
    return tokenizer(examples["text"], truncation=True, max_length=512)  # max_length can be reduced to e.g. 256 to increase speed, but long texts will be cut off

  dataset = dataset.map(tokenize, batched=True)

  # Set Training Arguments and Hyperparameter (bj)
  fp16_bool = True if torch.cuda.is_available() else False
  warmup_ratio = 0.25 if nli == True else 0.06

  print('\nSetting Training Arguments and Hyperparameter\n')
  train_args = TrainingArguments(
      num_train_epochs=epochs,  # this can be increased, but higher values increase training time. Good values for NLI are between 3 and 20.
      learning_rate=2e-5,
      per_device_train_batch_size=16,  # if you get an out-of-memory error, reduce this value to 8 or 4 and restart the runtime. Higher values increase training speed, but also increase memory requirements. Ideal values here are always a multiple of 8.
      per_device_eval_batch_size=64,  # if you get an out-of-memory error, reduce this value, e.g. to 40 and restart the runtime
      #gradient_accumulation_steps=2, # Can be used in case of memory problems to reduce effective batch size. accumulates gradients over X steps, only then backward/update. decreases memory usage, but also slightly speed. (!adapt/halve batch size accordingly)
      warmup_ratio=warmup_ratio,  # a good normal default value is 0.06 for normal BERT-base models, but since we want to reuse prior NLI knowledge and avoid catastrophic forgetting, we set the value higher
      weight_decay=0.1,
      seed=SEED_GLOBAL,
      load_best_model_at_end=True,
      metric_for_best_model="f1_micro", # changed (bj)
      fp16=fp16_bool,  # Can speed up training and reduce memory consumption, but only makes sense at batch-size > 8. loads two copies of model weights, which creates overhead. https://huggingface.co/transformers/performance.html?#fp16
      fp16_full_eval=fp16_bool,
      evaluation_strategy="epoch", # options: "no"/"steps"/"epoch"
      #eval_steps=10_000,  # evaluate after n steps if evaluation_strategy!='steps'. defaults to logging_steps
      save_strategy = "epoch",  # options: "no"/"steps"/"epoch"
      save_total_limit=5,             # If a value is passed, will limit the total amount of checkpoints. Deletes the older checkpoints in output_dir
      logging_strategy="epoch",
      report_to="all",  # "all"  # logging
      #push_to_hub=True,
      #hub_model_id=f"{model_short}_finetuned_{emotion}", # new parameter (bj)
      output_dir=f'./results/{emotion}/{model_short}',
      logging_dir=f'./logs/{emotion}/{model_short}',
  )

  # Train
  print("\nDownloading the Model\n")
  model = AutoModelForSequenceClassification.from_pretrained(model_name, config = config);

  print("\nTraining the Model\n")
  if nli == True:
    trainer = Trainer(
      model=model,
      tokenizer=tokenizer,
      args=train_args,
      train_dataset=dataset["train"],
      eval_dataset=dataset["test"],
      compute_metrics=lambda eval_pred: compute_metrics_nli_binary(eval_pred, label_text_alphabetical)
    )
  else:
    trainer = Trainer(
        model=model,
        tokenizer=tokenizer,
        args=train_args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        compute_metrics=lambda eval_pred: compute_metrics_standard(eval_pred)
    )

  trainer.train()

  return(trainer, model)


### inference
I created the following functions to consolidate some of the steps Laurer (2023) goes through and make it more applicable. I thought about creating one function, but the differences are signficant, so it is safer to have two functions. I mark my own additions with a #bj.

In [17]:
def inference(name, df_test, model_name, seed = None, nli = False, hypo_label_dict = {}):

  clean_memory()

  print('Initializing Tokenizer') #bj
  tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, model_max_length=512)

  print('\nInitializing Pipeline')
  if nli == True:
    if not hypo_label_dict: # https://stackoverflow.com/a/23177452/23292830
        raise Exception("No Dictionary") #https://www.w3schools.com/python/gloss_python_raise.asp
    hypothesis_lst = list(hypo_label_dict.values())

    pipe_classifier = pipeline(
      "zero-shot-classification",
      model= model_name,
      tokenizer=tokenizer,
      framework="pt",
      device=device,
      set_seed = seed
    )
  else:
    pipe_classifier = pipeline(
      "text-classification",
      model=model,  # if you have trained a model above, load_best_model_at_end in the training arguments has automatically replaced model with the fine-tuned model
      tokenizer=tokenizer,
      framework="pt",
      device=device,
  )

  # Create dummy dataset  #bj
  df_inference = df_test.copy(deep=True)
  text_lst = df_inference["text"].tolist()

  # use the pipeline with your chosen model for inference (prediction)
  print('\nPredicting ...') #bj

  if nli == True:
    pipe_output = pipe_classifier(
        text_lst,  # input any list of texts here
        candidate_labels=hypothesis_lst,
        hypothesis_template="{}",
        multi_label=False,  # here you can decide if, for your task, only one hypothesis can be true, or multiple can be true
        batch_size=32  # reduce this number to 8 or 16 if you get an out-of-memory error
    )

    # extract the predictions from pipe_outut
    hypothesis_pred_true_probability = []
    hypothesis_pred_true = []
    for dic in pipe_output:
      hypothesis_pred_true_probability.append(dic["scores"][0])
      hypothesis_pred_true.append(dic["labels"][0])

    # map the long hypotheses to their corresponding short label names
    hypothesis_label_dic_inference_inverted = {value: key for key, value in hypo_label_dict.items()}
    label_pred = [hypothesis_label_dic_inference_inverted[hypo] for hypo in hypothesis_pred_true]

    # add inference data to your original dataframe
    df_inference[f"{name}_pred"] = label_pred
    df_inference[f"{name}_prob"] = hypothesis_pred_true_probability

  else:
    pipe_output = pipe_classifier(
      text_lst,  # input any list of texts here
      batch_size=32  # reduce this number to 8 or 16 if you get an out-of-memory error
    )
    df_output = pd.DataFrame(pipe_output)

    df_inference[f"{name}_pred"] = df_output["label"].tolist()
    df_inference[f"{name}_prob"] = df_output["score"].round(2).tolist()


  # printing the classification report #bj
  print("\n")
  print(classification_report(df_inference["label_text"],  #bj
                              df_inference[f"{name}_pred"])) #bj

  return df_inference

# Analyses
I will try multiple models. I run them separately, because I am interested in seeing the individual results and possibly adjusting the number of epochs.  In general, I will perform hyperparameter tuning only for Naive Bayes, as it does not have much impact on transformer models according to Laurer (2023) and is too computationally intensive for this assignment. For the transformer modeels I might increase the number of epochs, if I notie that the peak performance is not reached during the current lengths of the training process.

## Nostalgia Analysis

### Naive Bayes
At first, I will use the Naive Bayes as 'classic' baseline (Atteveldt et al. 2021).

In [ ]:
nb = nb_training('nostalgia', nost_train)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

I uploaded the estimator from 18.05 to my GitHub, and it was downloaded through my initial download in this notebook. For comparability, I will use the downloaded version subsequently.

In [ ]:
# https://stackoverflow.com/a/71140866
#nb = load("/content/political_hope_nostalgia/nostalgia_nb_model.joblib")

In [ ]:
pred = nb.predict(nost_validation['text'])
print(classification_report(nost_validation['label_text'], pred))

               precision    recall  f1-score   support

    Nostalgia       0.74      0.79      0.76        47
Not Nostalgia       0.89      0.87      0.88        98

     accuracy                           0.84       145
    macro avg       0.82      0.83      0.82       145
 weighted avg       0.84      0.84      0.84       145



### Distilbert
At first, I will take the same model as Müller and Proksch and see whether I can replicate their performance

In [ ]:
model_name = "distilbert/distilbert-base-uncased"
emotion = 'nostalgia'
distilbert_trainer, distilbert_model= model_finetuning(emotion,
                      nost_train,
                      nost_validation,
                      model_name,
                      SEED_GLOBAL)
nostalgia_distilbert_results = distilbert_trainer.evaluate()


Data Preparation and Tokenizer Download



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]


 {'Not Nostalgia': 0, 'Nostalgia': 1} 



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Training the Model



Epoch,Training Loss,Validation Loss,Accuracy,Accuracy Balanced,F1 Macro,F1 Micro,Precision Macro,Recall Macro,Precision Micro,Recall Micro
1,0.550700,0.285802,0.910345,0.894920,0.897113,0.910345,0.899429,0.894920,0.910345,0.910345
2,0.259200,0.258339,0.917241,0.883413,0.900821,0.917241,0.927189,0.883413,0.917241,0.917241
3,0.185900,0.251321,0.917241,0.894485,0.903333,0.917241,0.914273,0.894485,0.917241,0.917241
4,0.121500,0.275797,0.924138,0.921733,0.914802,0.924138,0.908947,0.921733,0.924138,0.924138
5,0.073900,0.298087,0.924138,0.916196,0.913900,0.924138,0.911727,0.916196,0.924138,0.924138
6,0.047300,0.315235,0.924138,0.921733,0.914802,0.924138,0.908947,0.921733,0.924138,0.924138
7,0.028900,0.343951,0.917241,0.894485,0.903333,0.917241,0.914273,0.894485,0.917241,0.917241
8,0.017200,0.350716,0.917241,0.916630,0.907526,0.917241,0.900292,0.916630,0.917241,0.917241
9,0.010400,0.359145,0.917241,0.916630,0.907526,0.917241,0.900292,0.916630,0.917241,0.917241
10,0.007200,0.361178,0.924138,0.916196,0.913900,0.924138,0.911727,0.916196,0.924138,0.924138


Detailed metrics: 
               precision    recall  f1-score   support

           0       0.93      0.94      0.93        98
           1       0.87      0.85      0.86        47

    accuracy                           0.91       145
   macro avg       0.90      0.89      0.90       145
weighted avg       0.91      0.91      0.91       145
 

Detailed metrics: 
               precision    recall  f1-score   support

           0       0.91      0.98      0.94        98
           1       0.95      0.79      0.86        47

    accuracy                           0.92       145
   macro avg       0.93      0.88      0.90       145
weighted avg       0.92      0.92      0.92       145
 

Detailed metrics: 
               precision    recall  f1-score   support

           0       0.92      0.96      0.94        98
           1       0.91      0.83      0.87        47

    accuracy                           0.92       145
   macro avg       0.91      0.89      0.90       145
weighted a

Detailed metrics: 
               precision    recall  f1-score   support

           0       0.96      0.93      0.94        98
           1       0.86      0.91      0.89        47

    accuracy                           0.92       145
   macro avg       0.91      0.92      0.91       145
weighted avg       0.93      0.92      0.92       145
 



Because the best model was saved on huggingface and I need to preserve memory, I delete the model here.

In [ ]:
!rm -rf 'results/nostalgia/distilbert-base-uncased'
!rm -rf 'logs/nostalgia/distilbert-base-uncased'

### Deberta-v3
Deberta is the best openly available model according to Glue and SuperGlue Benchmarks. Therefore, I will try that as well

In [ ]:
model_name = "microsoft/deberta-v3-base"
emotion = 'nostalgia'
deberta_trainer, deberta_model= model_finetuning(emotion,
                      nost_train,
                      nost_validation,
                      model_name,
                      SEED_GLOBAL)
deberta_results = deberta_trainer.evaluate()


Data Preparation and Tokenizer Download



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]


 {'Not Nostalgia': 0, 'Nostalgia': 1} 



tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Training the Model



Epoch,Training Loss,Validation Loss,Accuracy,Accuracy Balanced,F1 Macro,F1 Micro,Precision Macro,Recall Macro,Precision Micro,Recall Micro
1,0.577700,0.402769,0.675862,0.500000,0.403292,0.675862,0.337931,0.500000,0.675862,0.675862
2,0.344000,0.263054,0.917241,0.911094,0.906572,0.917241,0.902530,0.911094,0.917241,0.917241
3,0.214800,0.252047,0.931034,0.921298,0.921298,0.931034,0.921298,0.921298,0.931034,0.931034
4,0.148400,0.455421,0.875862,0.902627,0.868182,0.875862,0.858982,0.902627,0.875862,0.875862
5,0.075100,0.280101,0.931034,0.915762,0.920400,0.931034,0.925556,0.915762,0.931034,0.931034
6,0.043900,0.404385,0.924138,0.910660,0.912941,0.924138,0.915349,0.910660,0.924138,0.924138
7,0.026400,0.451078,0.910345,0.900456,0.898246,0.910345,0.896155,0.900456,0.910345,0.910345
8,0.020100,0.504976,0.917241,0.900022,0.904480,0.917241,0.909444,0.900022,0.917241,0.917241
9,0.005700,0.506026,0.924138,0.910660,0.912941,0.924138,0.915349,0.910660,0.924138,0.924138
10,0.001300,0.515847,0.924138,0.910660,0.912941,0.924138,0.915349,0.910660,0.924138,0.924138


Detailed metrics: 
               precision    recall  f1-score   support

           0       0.68      1.00      0.81        98
           1       0.00      0.00      0.00        47

    accuracy                           0.68       145
   macro avg       0.34      0.50      0.40       145
weighted avg       0.46      0.68      0.55       145
 

Detailed metrics: 
               precision    recall  f1-score   support

           0       0.95      0.93      0.94        98
           1       0.86      0.89      0.88        47

    accuracy                           0.92       145
   macro avg       0.90      0.91      0.91       145
weighted avg       0.92      0.92      0.92       145
 

Detailed metrics: 
               precision    recall  f1-score   support

           0       0.95      0.95      0.95        98
           1       0.89      0.89      0.89        47

    accuracy                           0.93       145
   macro avg       0.92      0.92      0.92       145
weighted a

Detailed metrics: 
               precision    recall  f1-score   support

           0       0.95      0.95      0.95        98
           1       0.89      0.89      0.89        47

    accuracy                           0.93       145
   macro avg       0.92      0.92      0.92       145
weighted avg       0.93      0.93      0.93       145
 



In [ ]:
!rm -rf 'results/nostalgia/deberta-v3-base'
!rm -rf 'logs/nostalgia/deberta-v3-base'

### Zero-Shot Model
Zero-Shot Models have the advantage that I can add some context to the classification.

#### Zero-Shot

In [ ]:
nost_hypotheses = {
    "Nostalgia": "The text expresses nostalgia, it speaks positive about events and objects in the past or the past in general.",
    "Not Nostalgia": "The text does not express nostalgia."
    }
model = "MoritzLaurer/deberta-v3-base-zeroshot-v2.0"

dataset = inference(name = 'nost_zeroshot',
                    df_test = nost_train,
                    model_name = model,
                    seed = SEED_GLOBAL,
                    nli = True,
                    hypo_label_dict = nost_hypotheses)

Initializing Tokenizer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]


Initializing Pipeline


config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]


Predicting ...


               precision    recall  f1-score   support

    Nostalgia       0.74      0.20      0.32       372
Not Nostalgia       0.72      0.97      0.82       784

     accuracy                           0.72      1156
    macro avg       0.73      0.58      0.57      1156
 weighted avg       0.73      0.72      0.66      1156



#### Fine-Tuning

But as we saw with the results,  a bit learning is necessary nonetheless.

In [ ]:
zero_shot_trainer, zero_shot_model = model_finetuning('nostalgia',
                 nost_train,
                 nost_validation,
                 model,
                 SEED_GLOBAL,
                 epochs = 5,
                 nli = True,
                 hypo_label_dict = nost_hypotheses)



Data Preparation and Tokenizer Download

Length of df_train before formatting step: 1156.
After adding not_entailment training examples, the training data was augmented to 1900 texts.
Max augmentation could be: len(df_train) * 2 = 2312. It can also be lower, if there are more entail examples than not-entail for a majority class.
Number of hypotheses/classes:  2
Original test set size: 145
Test set size for NLI classification: 290



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/1900 [00:00<?, ? examples/s]

Map:   0%|          | 0/290 [00:00<?, ? examples/s]


Setting Training Arguments and Hyperparameter





/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Training the Model



Epoch,Training Loss,Validation Loss,Accuracy,Accuracy Balanced,F1 Macro,F1 Micro,Precision Macro,Recall Macro,Precision Micro,Recall Micro
1,0.449300,0.236001,0.917241,0.911094,0.906572,0.917241,0.902530,0.911094,0.917241,0.917241
2,0.192000,0.271922,0.931034,0.904690,0.918429,0.931034,0.937148,0.904690,0.931034,0.931034
3,0.072000,0.343779,0.937931,0.937473,0.930292,0.937931,0.924211,0.937473,0.937931,0.937931
4,0.020800,0.376859,0.937931,0.937473,0.930292,0.937931,0.924211,0.937473,0.937931,0.937931
5,0.009100,0.417102,0.931034,0.915762,0.920400,0.931034,0.925556,0.915762,0.931034,0.931034


Detailed metrics: 
               precision    recall  f1-score   support

           0       0.86      0.89      0.88        47
           1       0.95      0.93      0.94        98

    accuracy                           0.92       145
   macro avg       0.90      0.91      0.91       145
weighted avg       0.92      0.92      0.92       145
 

Detailed metrics: 
               precision    recall  f1-score   support

           0       0.95      0.83      0.89        47
           1       0.92      0.98      0.95        98

    accuracy                           0.93       145
   macro avg       0.94      0.90      0.92       145
weighted avg       0.93      0.93      0.93       145
 

Detailed metrics: 
               precision    recall  f1-score   support

           0       0.88      0.94      0.91        47
           1       0.97      0.94      0.95        98

    accuracy                           0.94       145
   macro avg       0.92      0.94      0.93       145
weighted a

In [ ]:
zero_shot_results = zero_shot_trainer.evaluate()
print(zero_shot_results)

Detailed metrics: 
               precision    recall  f1-score   support

           0       0.88      0.94      0.91        47
           1       0.97      0.94      0.95        98

    accuracy                           0.94       145
   macro avg       0.92      0.94      0.93       145
weighted avg       0.94      0.94      0.94       145
 

{'eval_loss': 0.3437848687171936, 'eval_accuracy': 0.9379310344827586, 'eval_accuracy_balanced': 0.9374728614850196, 'eval_f1_macro': 0.9302921852465147, 'eval_f1_micro': 0.9379310344827586, 'eval_precision_macro': 0.9242105263157895, 'eval_recall_macro': 0.9374728614850196, 'eval_precision_micro': 0.9379310344827586, 'eval_recall_micro': 0.9379310344827586, 'eval_runtime': 0.7514, 'eval_samples_per_second': 385.929, 'eval_steps_per_second': 6.654, 'epoch': 5.0}


In [ ]:
!rm -rf 'results/nostalgia/deberta-v3-base-zeroshot-v2.0'
!rm -rf 'logs/nostalgia/deberta-v3-base-zeroshot-v2.0'

### Evaluating the finetuned models
Now, I will evaluate the models on the test set.

In [ ]:
nost_nb_pred = nb.predict(nost_test['text'])
print(classification_report(nost_test['label_text'], pred))

               precision    recall  f1-score   support

    Nostalgia       0.30      0.33      0.31        46
Not Nostalgia       0.67      0.65      0.66        99

     accuracy                           0.54       145
    macro avg       0.49      0.49      0.49       145
 weighted avg       0.56      0.54      0.55       145



In [ ]:
model = "beja1996/distilbert-base-uncased_finetuned_nostalgia" # Getting the Fine-Tuned Model from Huggingface

nost_distilbert_eval = inference('distilbert', nost_test, model)

Initializing Tokenizer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]


Initializing Pipeline


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]


Predicting ...


               precision    recall  f1-score   support

    Nostalgia       0.91      0.89      0.90        46
Not Nostalgia       0.95      0.96      0.95        99

     accuracy                           0.94       145
    macro avg       0.93      0.93      0.93       145
 weighted avg       0.94      0.94      0.94       145



In [ ]:
model = "beja1996/deberta-v3-base_finetuned_nostalgia" # Getting the Fine-Tuned Model from Huggingface

nost_deberta_eval = inference('deberta', nost_test, model)

Initializing Tokenizer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]


Initializing Pipeline


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]


Predicting ...


               precision    recall  f1-score   support

    Nostalgia       0.84      0.93      0.89        46
Not Nostalgia       0.97      0.92      0.94        99

     accuracy                           0.92       145
    macro avg       0.91      0.93      0.91       145
 weighted avg       0.93      0.92      0.93       145



In [ ]:
model = "beja1996/deberta-v3-base-zeroshot-v2.0_finetuned_nostalgia" # Getting the Fine-Tuned Model from Huggingface

nost_zeroshot_eval = inference('zeroshot', nost_test, model, SEED_GLOBAL, True, nost_hypotheses)

Initializing Tokenizer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(



Initializing Pipeline

Predicting ...


               precision    recall  f1-score   support

    Nostalgia       0.88      0.91      0.89        46
Not Nostalgia       0.96      0.94      0.95        99

     accuracy                           0.93       145
    macro avg       0.92      0.93      0.92       145
 weighted avg       0.93      0.93      0.93       145



The following code is not done with a function, because it would have taken me more time to create a function than writing it once for hope and once for nostalgia.

In [ ]:
inferences = nost_test.copy()
inferences['nb_pred'] = nost_nb_pred
inferences['nb_prob'] =  nb.predict_proba(nost_test['text'])[:,1].round(2)
inferences = pd.concat([inferences,
           nost_distilbert_eval[['distilbert_pred', 'distilbert_prob']],
           nost_deberta_eval[['deberta_pred', 'deberta_prob']],
           nost_zeroshot_eval[['zeroshot_pred', 'zeroshot_prob']]
           ],
           axis= 1)

In [ ]:
models = ['nb', 'distilbert', 'deberta', 'zeroshot']
table = []
for model in models:
  metrics = metrics_core(inferences['label_text'], inferences[f'{model}_pred'])
  table.append(metrics)
nost_metrics = pd.DataFrame(table).round(2)
nost_metrics.index = models
nost_metrics

,accuracy,accuracy_balanced,f1_macro,f1_micro,precision_macro,recall_macro,precision_micro,recall_micro
nb,0.86,0.84,0.84,0.86,0.84,0.84,0.86,0.86
distilbert,0.94,0.93,0.93,0.94,0.93,0.93,0.94,0.94
deberta,0.92,0.93,0.91,0.92,0.91,0.93,0.92,0.92
zeroshot,0.93,0.93,0.92,0.93,0.92,0.93,0.93,0.93


In [ ]:
nost_metrics.to_csv('nostalgia_metrics.csv')
files.download('nostalgia_metrics.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Hope Analysis

### Naive Bayes
At first, I will use the Naive Bayes as 'classic' baseline (Atteveldt et al. 2021).

In [ ]:
nb = nb_training('hope', hope_train)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


I uploaded the estimator from 18.05 to my GitHub, and it was downloaded through my initial download in this notebook. For comparability, I will use the downloaded version subsequently.

In [ ]:
# https://stackoverflow.com/a/71140866
#nb = load("/content/political_hope_nostalgia/hope_nb_model.joblib")

In [ ]:
pred = nb.predict(hope_validation['text'])
print(classification_report(hope_validation['label_text'], pred))

              precision    recall  f1-score   support

        Hope       0.76      0.75      0.75       311
    Not Hope       0.75      0.76      0.75       309

    accuracy                           0.75       620
   macro avg       0.75      0.75      0.75       620
weighted avg       0.75      0.75      0.75       620



### Distilbert

In [ ]:
model_name = "distilbert/distilbert-base-uncased"
emotion = 'hope'
distilbert_trainer, distilbert_model= model_finetuning(emotion,
                      hope_train,
                      hope_validation,
                      model_name,
                      SEED_GLOBAL)
hope_distilbert_results = distilbert_trainer.evaluate()


Data Preparation and Tokenizer Download



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]


 {'Not Hope': 0, 'Hope': 1} 



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Training the Model



Epoch,Training Loss,Validation Loss,Accuracy,Accuracy Balanced,F1 Macro,F1 Micro,Precision Macro,Recall Macro,Precision Micro,Recall Micro
1,0.502600,0.394708,0.843296,0.843251,0.843163,0.843296,0.844351,0.843251,0.843296,0.843296
2,0.284000,0.509495,0.830372,0.830327,0.830228,0.830372,0.831385,0.830327,0.830372,0.830372
3,0.175400,0.511835,0.836834,0.836789,0.836696,0.836834,0.837868,0.836789,0.836834,0.836834
4,0.094900,0.739847,0.833603,0.833542,0.833352,0.833603,0.835475,0.833542,0.833603,0.833603
5,0.066100,1.029637,0.789984,0.789764,0.785948,0.789984,0.813102,0.789764,0.789984,0.789984
6,0.038200,0.867915,0.827141,0.827158,0.827124,0.827141,0.827302,0.827158,0.827141,0.827141
7,0.026500,0.965335,0.823910,0.823865,0.823761,0.823910,0.824903,0.823865,0.823910,0.823910
8,0.020300,1.066496,0.809370,0.809260,0.808446,0.809370,0.815203,0.809260,0.809370,0.809370
9,0.013500,1.062382,0.820679,0.820634,0.820527,0.820679,0.821661,0.820634,0.820679,0.820679
10,0.006600,1.103944,0.814216,0.814146,0.813836,0.814216,0.816645,0.814146,0.814216,0.814216


Detailed metrics: 
               precision    recall  f1-score   support

           0       0.86      0.82      0.84       309
           1       0.83      0.87      0.85       310

    accuracy                           0.84       619
   macro avg       0.84      0.84      0.84       619
weighted avg       0.84      0.84      0.84       619
 

Detailed metrics: 
               precision    recall  f1-score   support

           0       0.85      0.80      0.83       309
           1       0.81      0.86      0.84       310

    accuracy                           0.83       619
   macro avg       0.83      0.83      0.83       619
weighted avg       0.83      0.83      0.83       619
 

Detailed metrics: 
               precision    recall  f1-score   support

           0       0.86      0.81      0.83       309
           1       0.82      0.86      0.84       310

    accuracy                           0.84       619
   macro avg       0.84      0.84      0.84       619
weighted a

Detailed metrics: 
               precision    recall  f1-score   support

           0       0.86      0.82      0.84       309
           1       0.83      0.87      0.85       310

    accuracy                           0.84       619
   macro avg       0.84      0.84      0.84       619
weighted avg       0.84      0.84      0.84       619
 



Because the best model was saved on huggingface and I need to preserve memory, I delete the model here.

In [ ]:
!rm -rf 'results/hope/distilbert-base-uncased'
!rm -rf 'logs/hope/distilbert-base-uncased'

### Deberta-v3
Deberta is the best openly available model according to Glue and SuperGlue Benchmarks. Therefore, I will try that as well

In [ ]:
model_name = "microsoft/deberta-v3-base"
emotion = 'hope'
deberta_trainer, deberta_model= model_finetuning(emotion,
                      hope_train,
                      hope_validation,
                      model_name,
                      SEED_GLOBAL)
deberta_results = deberta_trainer.evaluate()


Data Preparation and Tokenizer Download



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]


 {'Not Hope': 0, 'Hope': 1} 



tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Training the Model



Epoch,Training Loss,Validation Loss,Accuracy,Accuracy Balanced,F1 Macro,F1 Micro,Precision Macro,Recall Macro,Precision Micro,Recall Micro
1,0.536800,0.431567,0.820679,0.820644,0.820587,0.820679,0.821254,0.820644,0.820679,0.820679
2,0.320700,0.461031,0.836834,0.836763,0.836499,0.836834,0.839442,0.836763,0.836834,0.836834
3,0.227900,0.470577,0.854604,0.854583,0.854573,0.854604,0.854850,0.854583,0.854604,0.854604
4,0.168800,0.630187,0.848142,0.848090,0.847967,0.848142,0.849623,0.848090,0.848142,0.848142
5,0.118700,0.842438,0.815832,0.815633,0.812937,0.815832,0.836170,0.815633,0.815832,0.815832
6,0.074000,0.752476,0.838449,0.838485,0.838378,0.838449,0.839130,0.838485,0.838449,0.838449
7,0.058200,0.933431,0.822294,0.822200,0.821657,0.822294,0.826743,0.822200,0.822294,0.822294


Detailed metrics: 
               precision    recall  f1-score   support

           0       0.83      0.80      0.82       309
           1       0.81      0.84      0.82       310

    accuracy                           0.82       619
   macro avg       0.82      0.82      0.82       619
weighted avg       0.82      0.82      0.82       619
 

Detailed metrics: 
               precision    recall  f1-score   support

           0       0.87      0.79      0.83       309
           1       0.81      0.88      0.84       310

    accuracy                           0.84       619
   macro avg       0.84      0.84      0.84       619
weighted avg       0.84      0.84      0.84       619
 

Detailed metrics: 
               precision    recall  f1-score   support

           0       0.86      0.84      0.85       309
           1       0.85      0.87      0.86       310

    accuracy                           0.85       619
   macro avg       0.85      0.85      0.85       619
weighted a

In [ ]:
!rm -rf 'results/hope/deberta-v3-base'
!rm -rf 'logs/hope/deberta-v3-base'

### Zero-Shot Model
Zero-Shot Models have the advantage that I can add some context to the classification.

#### Zero-Shot

In [ ]:
hope_hypotheses = {
    "Hope": "The text expresses hope, a future-oriented expectation, desire or wish towards a general or specific event.",
    "Not Hope": "The text does not express hope, wish, desire, or future-oriented expectation."
}
model = "MoritzLaurer/deberta-v3-base-zeroshot-v2.0"

dataset = inference(name = 'hope_zeroshot',
                    df_test = hope_train,
                    model_name = model,
                    seed = SEED_GLOBAL,
                    nli = True,
                    hypo_label_dict = hope_hypotheses)

Initializing Tokenizer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]


Initializing Pipeline


config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]


Predicting ...


              precision    recall  f1-score   support

        Hope       0.79      0.56      0.66      2483
    Not Hope       0.66      0.85      0.74      2470

    accuracy                           0.71      4953
   macro avg       0.73      0.71      0.70      4953
weighted avg       0.73      0.71      0.70      4953



#### Fine-Tuning

But as we saw with the results,  a bit learning is necessary nonetheless.

In [ ]:
zero_shot_trainer, zero_shot_model = model_finetuning('hope',
                 hope_train,
                 hope_validation,
                 model,
                 SEED_GLOBAL,
                 epochs = 5,
                 nli = True,
                 hypo_label_dict = hope_hypotheses)


Data Preparation and Tokenizer Download

Length of df_train before formatting step: 4953.
After adding not_entailment training examples, the training data was augmented to 9893 texts.
Max augmentation could be: len(df_train) * 2 = 9906. It can also be lower, if there are more entail examples than not-entail for a majority class.
Number of hypotheses/classes:  2
Original test set size: 620
Test set size for NLI classification: 1240



/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]


Training the Model



Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,Accuracy Balanced,F1 Macro,F1 Micro,Precision Macro,Recall Macro,Precision Micro,Recall Micro
1,0.447100,0.392781,0.850000,0.850191,0.849520,0.850000,0.854974,0.850191,0.850000,0.850000
2,0.265800,0.498012,0.856452,0.856528,0.856388,0.856452,0.857253,0.856528,0.856452,0.856452
3,0.119400,0.669816,0.859677,0.859681,0.859677,0.859677,0.859677,0.859681,0.859677,0.859677
4,0.045600,0.898129,0.859677,0.859619,0.859616,0.859677,0.860155,0.859619,0.859677,0.859677
5,0.023600,0.998234,0.861290,0.861268,0.861277,0.861290,0.861358,0.861268,0.861290,0.861290


Detailed metrics: 
               precision    recall  f1-score   support

           0       0.90      0.79      0.84       311
           1       0.81      0.91      0.86       309

    accuracy                           0.85       620
   macro avg       0.85      0.85      0.85       620
weighted avg       0.86      0.85      0.85       620
 

Detailed metrics: 
               precision    recall  f1-score   support

           0       0.88      0.83      0.85       311
           1       0.84      0.88      0.86       309

    accuracy                           0.86       620
   macro avg       0.86      0.86      0.86       620
weighted avg       0.86      0.86      0.86       620
 

Detailed metrics: 
               precision    recall  f1-score   support

           0       0.86      0.86      0.86       311
           1       0.86      0.86      0.86       309

    accuracy                           0.86       620
   macro avg       0.86      0.86      0.86       620
weighted a

In [ ]:
zero_shot_results = zero_shot_trainer.evaluate()
print(zero_shot_results)

Detailed metrics: 
               precision    recall  f1-score   support

           0       0.86      0.87      0.86       311
           1       0.87      0.85      0.86       309

    accuracy                           0.86       620
   macro avg       0.86      0.86      0.86       620
weighted avg       0.86      0.86      0.86       620
 

{'eval_loss': 0.9982327222824097, 'eval_accuracy': 0.8612903225806452, 'eval_accuracy_balanced': 0.861268067305591, 'eval_f1_macro': 0.8612773308634523, 'eval_f1_micro': 0.8612903225806452, 'eval_precision_macro': 0.8613583138173302, 'eval_recall_macro': 0.861268067305591, 'eval_precision_micro': 0.8612903225806452, 'eval_recall_micro': 0.8612903225806452, 'eval_runtime': 2.508, 'eval_samples_per_second': 494.426, 'eval_steps_per_second': 7.975, 'epoch': 5.0}


In [ ]:
!rm -rf '/content/results/hope/deberta-v3-base-zeroshot-v2.0'
!rm -rf '/content/logs/hope/deberta-v3-base-zeroshot-v2.0'

### Evaluating the finetuned models
Now, I will evaluate the models on the test set.

In [19]:
# https://stackoverflow.com/a/71140866
nb = load("/content/political_hope_nostalgia/hope_nb_model.joblib")

In [20]:
hope_nb_pred = nb.predict(hope_test['text'])
print(classification_report(hope_test['label_text'], hope_nb_pred))

              precision    recall  f1-score   support

        Hope       0.82      0.80      0.81       310
    Not Hope       0.81      0.82      0.82       308

    accuracy                           0.81       618
   macro avg       0.81      0.81      0.81       618
weighted avg       0.81      0.81      0.81       618



In [18]:
model = "beja1996/distilbert-base-uncased_finetuned_hope" # Getting the Fine-Tuned Model from Huggingface

hope_distilbert_eval = inference('distilbert', hope_test, model)

Initializing Tokenizer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]


Initializing Pipeline


config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]


Predicting ...


              precision    recall  f1-score   support

        Hope       0.95      0.97      0.96       310
    Not Hope       0.97      0.95      0.96       308

    accuracy                           0.96       618
   macro avg       0.96      0.96      0.96       618
weighted avg       0.96      0.96      0.96       618



In [21]:
model = "beja1996/deberta-v3-base_finetuned_hope" # Getting the Fine-Tuned Model from Huggingface

hope_deberta_eval = inference('deberta', hope_test, model)

Initializing Tokenizer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]


Initializing Pipeline


config.json:   0%|          | 0.00/992 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]


Predicting ...


              precision    recall  f1-score   support

        Hope       0.96      0.96      0.96       310
    Not Hope       0.96      0.96      0.96       308

    accuracy                           0.96       618
   macro avg       0.96      0.96      0.96       618
weighted avg       0.96      0.96      0.96       618



In [23]:
hope_hypotheses = {
    "Hope": "The text expresses hope, a future-oriented expectation, desire or wish towards a general or specific event.",
    "Not Hope": "The text does not express hope, wish, desire, or future-oriented expectation."
}
model = "beja1996/deberta-v3-base-zeroshot-v2.0_finetuned_hope" # Getting the Fine-Tuned Model from Huggingface

hope_zeroshot_eval = inference('zeroshot', hope_test, model, SEED_GLOBAL, True, hope_hypotheses)

Initializing Tokenizer


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.65M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]


Initializing Pipeline


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]


Predicting ...


              precision    recall  f1-score   support

        Hope       0.96      0.95      0.95       310
    Not Hope       0.95      0.96      0.95       308

    accuracy                           0.95       618
   macro avg       0.95      0.95      0.95       618
weighted avg       0.95      0.95      0.95       618



In [24]:
inferences = hope_test.copy()
inferences['nb_pred'] = hope_nb_pred
inferences['nb_prob'] =  nb.predict_proba(hope_test['text'])[:,1].round(2)
inferences = pd.concat([inferences,
           hope_distilbert_eval[['distilbert_pred', 'distilbert_prob']],
           hope_deberta_eval[['deberta_pred', 'deberta_prob']],
           hope_zeroshot_eval[['zeroshot_pred', 'zeroshot_prob']]
           ],
           axis= 1)

In [25]:
models = ['nb', 'distilbert', 'deberta', 'zeroshot']
table = []
for model in models:
  metrics = metrics_core(inferences['label_text'], inferences[f'{model}_pred'])
  table.append(metrics)
hope_metrics = pd.DataFrame(table).round(2)
hope_metrics.index = models
hope_metrics

,accuracy,accuracy_balanced,f1_macro,f1_micro,precision_macro,recall_macro,precision_micro,recall_micro
nb,0.81,0.81,0.81,0.81,0.81,0.81,0.81,0.81
distilbert,0.96,0.96,0.96,0.96,0.96,0.96,0.96,0.96
deberta,0.96,0.96,0.96,0.96,0.96,0.96,0.96,0.96
zeroshot,0.95,0.95,0.95,0.95,0.95,0.95,0.95,0.95


The values are higher than those of the validation set. This should not be the case. Is the test set in the training set?

In [31]:
print(f"There are {sum(inferences['text'].isin(hope_train['text']))} texts from the test set in the training set")

There are 0 texts from the test set in the training set


In [32]:
print(f"There are {sum(inferences['text'].isin(hope_validation['text']))} texts from the test set in the validation set")

There are 0 texts from the test set in the validation set


Maybe there are certain formulations IN the training set, which make the prediction quite easy and those are also often in the validation set, inflating the accuracy. Unfortunately, I can not solve the issue here.

In [ ]:
hope_metrics.to_csv('hope_metrics.csv')
files.download('hope_metrics.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# References
- Balouchzahi, F., Sidorov, G., & Gelbukh, A. (2023). PolyHope: Two-level hope speech detection from tweets. Expert Systems with Applications, 225, 120078. https://doi.org/10.1016/j.eswa.2023.120078
- Laurer, M. (2023, August 22). Fine-tuning BERT-NLI. Data Science Summer School 2023, Berlin. https://github.com/MoritzLaurer/summer-school-transformers-2023/blob/main/4_tune_bert_nli.ipynb
- Müller, S., & Proksch, S.-O. (2023). Nostalgia in European Party Politics: A Text-Based Measurement Approach. British Journal of Political Science, 1–13. doi:10.1017/S0007123423000571
- Müller, S., & Proksch, S.-O. (2023). PolNos: Political nostalgia in party manifestos \[Data set]. doi:10.7910/DVN/L198GI